# Large problem with 1000 assets and 100 factors

In [ ]:
import numpy as np
import pandas as pd
import cvxpy as cvx

from cvx.risk.factor import FactorModel
from cvx.risk.linalg import cholesky
from util.random import random_factors, random_beta, random_noise, random_assets

In [ ]:
T = 2000
factors = random_factors(T=T, N=100, const_factor=False)

In [ ]:
beta = random_beta(assets=random_assets(1000), factors=factors)

In [ ]:
ret = factors @ beta + 0.01 * random_noise(factors @ beta)

In [ ]:
triangle = FactorModel(assets=len(ret.columns), k=100)

In [ ]:
#w = triangle.weights
w = cvx.Variable(1000)
y = cvx.Variable(100)

objective = cvx.Minimize(triangle.estimate_risk(w, y=y))
constraints = [w >= 0, cvx.sum(w) == 1, y==triangle.exposure @ w]

triangle.exposure.values = beta
triangle.chol.values = cholesky(factors.cov().values)
triangle.idiosyncratic_risk.value = pd.DataFrame(data=ret - factors @ beta, index=ret.index, columns=ret.columns).std().values

In [ ]:
%%timeit
for i in range(1):
    objective = cvx.Minimize(triangle.estimate_risk(w, y=y))
    constraints = [w >= 0, cvx.sum(w) == 1, y==triangle.exposure @ w]

    problem = cvx.Problem(objective=objective, constraints=constraints)
    triangle.exposure.values = beta
    triangle.chol.values = cholesky(factors.cov().values)
    triangle.idiosyncratic_risk.value = pd.DataFrame(data=ret - factors @ beta, index=ret.index, columns=ret.columns).std().values

    problem.solve(ignore_dpp=True)

In [ ]:
# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)

In [ ]:
%%timeit
for i in range(1):
    objective = cvx.Minimize(triangle.estimate_risk(w, y=y))
    constraints = [w >= 0, cvx.sum(w) == 1, y==triangle.exposure @ w]

    problem = cvx.Problem(objective=objective, constraints=constraints)
    triangle.exposure.values = beta
    triangle.chol.values = cholesky(factors.cov().values)
    triangle.idiosyncratic_risk.value = pd.DataFrame(data=ret - factors @ beta, index=ret.index, columns=ret.columns).std().values

    problem.solve(ignore_dpp=True)

In [ ]:
# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)

In [ ]:
# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)

In [ ]:
# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)

In [ ]:
# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)